# ALGORITHMS FOR DATA SCIENCE (by (Prof. Dmitrii Bakhitov)
# MapReduce Assignment
## CRN:-CS-676 (74060) ; FALL 2023
## Submitted by: Piyush Gupta

### Objective:
**Apply MapReduce concepts in a real-world data processing scenario using Python.**

In [18]:
pip install PyPDF2

In [19]:
# Importing necessary libraries
import PyPDF2
import re
from collections import defaultdict
from multiprocessing import Pool
import time

PyPDF2 for PDF processing, re for regular expressions, defaultdict from collections for easier data management, Pool from multiprocessing for parallel processing, and time for tracking execution times.

### Step 1: Importing the Dataset
First, import the necessary libraries and read the content of the PDF file. This will serve as our dataset for the MapReduce task.


In [20]:
pdf_path = 'What Is a Statistical Project.pdf'

try:
    # Open the PDF file
    with open(pdf_path, 'rb') as file:
        pdf_reader = PyPDF2.PdfReader(file)
        num_pages = len(pdf_reader.pages)

        # Extract text from each page and combine it
        text_data = ""
        for page in range(num_pages):
            text_data += pdf_reader.pages[page].extract_text() + "\n"
except FileNotFoundError:
    print(f"File not found: {pdf_path}")

In [21]:
text_data

'A statistical project is the process of answering \na research question using statistical techniques and presenting the work in a written report. Th  e \nresearch question may arise from any fi  eld of \nscientifi  c endeavor, such as athletics, advertising, \naerodynamics, or nutrition. A project diff  ers \nfrom a statistical poster in that a written report is used to present the fi  ndings.\nData-Based Problemsolving\nTh e process of developing a statistical project \nshould demonstrate the scientifi  c method and \npose a focused question or questions, collect appropriate data, analyze the data thoughtfully, and draw correct conclusions.\nBecause students are asking questions continually \nabout what touches their lives, they should have little trouble generating questions about them-selves or their schools, families, neighborhoods, or interesting phenomena in the world.\nOnce a question is proposed, students should \nexamine it. First, is it a question that can be answered? (Th  

In [22]:
len(text_data)

4632

### Step 2: Problem Statement

Perform a word count analysis on the text data extracted from the PDF. We will count the frequency of each word using the MapReduce model.

### Step 3: Implement MapReduce

Implement the MapReduce model with custom map and reduce functions.

In [23]:
# Map Function
def map_function(text):
    # Split the text into words and map each word to a count of 1
    return [(word, 1) for word in re.findall(r'\w+', text.lower())]

# Reduce Function
def reduce_function(mapped_items):
    # Reduce the mapped items by summing the counts of each word
    reduced = defaultdict(int)
    for word, count in mapped_items:
        reduced[word] += count
    return dict(reduced)

# Parallel MapReduce Function
def parallel_map_reduce(data, map_func, reduce_func, pool_size):
    # Use a pool of workers to apply the map function in parallel
    with Pool(pool_size) as pool:
        map_results = pool.map(map_func, data)
        # Flatten the list of lists to a single list of key-value pairs
        flattened_map_results = [item for sublist in map_results for item in sublist]
        # Reduce the mapped data
        reduced_results = reduce_func(flattened_map_results)
    return reduced_results

### Step 4: Performance Analysis
Run the MapReduce job and measure the execution time for each.

In [24]:
# Function to run performance analysis with varying pool sizes
def performance_analysis(data, map_func, reduce_func, pool_sizes):
    times = {}
    for size in pool_sizes:
        start_time = time.time()
        parallel_map_reduce(data, map_func, reduce_func, pool_size=size)
        end_time = time.time()
        times[size] = end_time - start_time
    return times

In [25]:
#  Proceed with MapReduce and performance analysis
if 'text_data' in locals():
    # Splitting the text data into chunks for parallel processing
    data_chunks = text_data.split('\n')

    # Running the performance analysis with varying pool sizes
    pool_sizes = [1, 2, 4, 8]
    performance_times = performance_analysis(data_chunks, map_function, reduce_function, pool_sizes)

    # Measuring execution time for the non-parallelized approach
    start_time_non_parallel = time.time()
    word_counts_non_parallel = reduce_function(map_function(text_data))
    end_time_non_parallel = time.time()
    execution_time_non_parallel = end_time_non_parallel - start_time_non_parallel

    # Adding non-parallelized execution time to the performance times dictionary
    performance_times['non_parallel'] = execution_time_non_parallel

    # Displaying performance times
    print("Performance Analysis Times:")
    print(performance_times)

Performance Analysis Times:
{1: 0.028107404708862305, 2: 0.027884244918823242, 4: 0.048342227935791016, 8: 0.12561392784118652, 'non_parallel': 0.0005130767822265625}


### Step 5: Report Writing
Document the approach, code, results, and observations. Discuss the scalability and efficiency of the MapReduce model based on the findings, and reflect on the challenges faced and potential improvements.

### Report Writing for MapReduce Performance Analysis

## Approach
The approach taken in this MapReduce job involves a parallelized computation to perform a word count on a large text dataset. The code utilizes Python's multiprocessing capabilities to distribute the map and reduce tasks across multiple worker processes. The performance of this parallelized approach is compared with a non-parallelized (sequential) execution. The objective is to understand the impact of process pooling on the execution time of the MapReduce job.

## Results
The results displayed in the output are as follows:

Performance Analysis Times:

Performance Analysis Times:
{1: 0.028107404708862305, 2: 0.027884244918823242, 4: 0.048342227935791016, 8: 0.12561392784118652, 'non_parallel': 0.0005130767822265625}

## Observations
From the output, the following observations can be made:

- The non-parallelized approach has the fastest execution time at approximately `0.00051` seconds.
- With a single process in the pool (which effectively makes it sequential), the execution time is around `0.028` seconds, which is slower than the non-parallelized run.
- As the pool size increases to 2, the execution time decreases slightly to `0.0278` seconds, indicating that some benefit is gained from parallel processing.
- However, further increasing the pool size to 4 and 8 results in increased execution times, `0.0483` and `0.125` seconds, respectively. This suggests that there is an overhead associated with managing a larger number of processes that outweighs the benefits of parallelism for this task.

## Scalability and Efficiency
The MapReduce model typically exhibits improved performance with increased parallelism, but the results indicate an inverse relationship between pool size and performance in this specific case. This could be due to several factors such as overhead from process creation and management, the overhead of inter-process communication, or the small size of the dataset which does not require extensive parallel processing.

Efficiency in parallel computing is not solely about increasing worker count; it's also about the overhead and the nature of the task. The results suggest that for smaller datasets, or tasks with a quick execution time, the overhead of parallelism can outweigh its benefits.

## Challenges Faced
The challenges that might have been faced include:

- Determining the optimal number of processes for the pool.
- Managing the overhead associated with parallel processing.
- Ensuring that the task is suitable for parallel execution and that the dataset is large enough to benefit from it.

## Potential Improvements
To improve the performance of the MapReduce job, one could:

- Conduct thorough profiling to understand where the overhead is coming from.
- Experiment with different chunk sizes for the dataset to optimize the amount of data processed per worker.
- Implement a more sophisticated mechanism for dynamically adjusting the pool size based on the complexity of the task and the size of the dataset.

In conclusion, while the MapReduce model is powerful for processing large datasets, it requires careful tuning and consideration of overheads to achieve optimal performance. This case study highlights the importance of understanding the computational environment and the characteristics of the task at hand when designing parallelized solutions.
